In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Input

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import chi2
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import KFold

# Eksploratory Data Analysis (EDA) 
**Pendekatan Untuk Menganalisis Set Data**

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data.head()

# Sebaran Data Film dan TV Show Di Netflix

In [ ]:
bar, ax = plt.subplots(figsize = (12,12))
plt.pie(data['type'].value_counts(), labels = data['type'].value_counts().index, autopct="%.1f%%")
plt.title('Sebaran Data Film dan Tv Show di Netflix', size=20)

# Banyaknya Rilis Dalam Tahun

In [ ]:
bar, ax = plt.subplots(figsize = (10,10))
sns.barplot(x = data['release_year'].value_counts().index[:5], y = data['release_year'].value_counts()[:5])
plt.xlabel('Tahun')
plt.ylabel('Banyaknya')
plt.title('Banyaknya Rilis Dalam Tahun')

In [ ]:
movie_data = data[data['type'] == 'Movie']
tv_show_data = data[data['type'] == 'TV Show']
# bar,ax = plt.subplots(1,2,figsize=(10,10))
temp = data[['type', 'release_year']]
temp = temp.value_counts().to_frame()
temp.reset_index(level=[0,1], inplace=True)
temp = temp.rename(columns = {0:'count'})
temp = pd.concat([temp[temp['type'] == 'Movie'][:5], temp[temp['type']== 'TV Show'][:5]])

In [ ]:
# ax, bar = plt.subplots(figsize = (10,10))
sns.catplot(x = 'release_year', y = 'count', hue = 'type', data = temp, kind = 'point')
plt.xlabel('Tahun Rilis')
plt.ylabel('Banyaknya')
plt.title('Banyaknya Rilis pertahun dalam film dan Tv Show', size=14)

# Genre Paling Populer dalam 10 Tahun

In [ ]:
temp = list()
clean_data = data.dropna()
clean_data.reset_index(inplace=True)
for ind, element in clean_data.iterrows():
    type_show = element['release_year']
    for cast in str(element['listed_in']).split(','):
        temp.append([type_show, cast])
cast_data = pd.DataFrame(temp, columns= ['release_year', 'cast'])
cast_data

In [ ]:
cast = cast_data.value_counts().to_frame()
cast.reset_index(level=[0,1], inplace=True)
cast = cast.rename(columns = {0:'count'})


years = [2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010]
year_data = list()
for year in years:
    temp1 = cast[cast['release_year'] == year].iloc[0,:]
    temp2 = cast[cast['release_year'] == year].iloc[1,:]
    year_data.append(list(temp1))
    year_data.append(list(temp2))
    
year = pd.DataFrame(year_data, columns=('years', 'genre', 'count'))
bar, ax = plt.subplots(figsize=(10,10))
sns.barplot(x = 'years', y ='count', hue='genre', data = year)
plt.xlabel('Tahun')
plt.ylabel('Banyaknya')
plt.title('Genre Paling Populer dalam 10 Tahun', size=20)

# Predisksi Rating
**Menggunakan Algoritma Naive Bayes**

In [ ]:
def get_data():
    movies = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
    
    # Replace NaN values with empty string
    movies.dropna(inplace=True)
    
    # Uniformisation of labels with same signification
    movies['rating'].replace(to_replace='PG-13', value='TV-PG', inplace=True)
    movies['rating'].replace(to_replace='PG', value='TV-PG', inplace=True)
    movies['rating'].replace(to_replace='TV-Y7-FV', value='TV-Y7', inplace=True)
    movies['rating'].replace(to_replace='G', value='TV-G', inplace=True)
    movies['rating'].replace(to_replace='NC-17', value='R', inplace=True)
    # Drop rows that don't have any labels (NR = UR = Unrated)
    movies.drop(movies[movies['rating']=='NR'].index, inplace=True)
    movies.drop(movies[movies['rating']==''].index, inplace=True)
    movies.drop(movies[movies['rating']=='UR'].index, inplace=True)
    #Drop useless labels
    movies.drop(movies[movies['rating']=='TV-G'].index, inplace=True)
    movies.drop(movies[movies['rating']=='TV-Y'].index, inplace=True)
    movies.drop(movies[movies['rating']=='TV-Y7'].index, inplace=True)


    #Drop useless colmns
    movies = movies[["type", "title", "description", "director", "cast", 'rating', "listed_in", "country"]]
    
    return movies

# Prediksi Menggunakan Algoritma Naive Bayes

In [ ]:
movies = get_data()

#Plot label repartition
fig = plt.figure(figsize=(8,6))
movies.groupby('rating').title.count().plot.bar(ylim=0)
plt.show()

In [ ]:
#Convert labels to int and create a new column
movies['category_id'] = movies['rating'].factorize()[0]
#Get unique values
category_id_df = movies[["rating", "category_id"]].drop_duplicates().sort_values('category_id')
#Dicts associating rating and its int value
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'rating']].values)
labels = movies.category_id

#Group all the columns together
movies ['txt'] = movies['type'] + " " + movies['title'] + " " + movies['description'] + " " + movies['listed_in'] 
movies['txt'] = movies['txt'] + " " + movies['director'] + " " + movies['cast'] + " " + movies["country"]
movies = movies[['rating', 'txt']]

#Cross val
mskf = KFold(n_splits = 15, random_state=0, shuffle = True)
acc = []

for train_index, test_index in mskf.split(movies['txt']) :
    X_train, X_test = movies['txt'].iloc[train_index], movies['txt'].iloc[test_index]
    y_train, y_test = movies['rating'].iloc[train_index], movies['rating'].iloc[test_index]
    
    #Preprocess
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    #Fit
    clf = MultinomialNB().fit(X_train_tfidf, y_train)
    
    #score
    acc.append(clf.score(count_vect.transform(X_test.values), y_test.values))
    
plot_confusion_matrix(clf, count_vect.transform(X_test), np.array(y_test.values))
plt.show()

In [ ]:
print("Accuracy with Naive Bayes: " + str(np.mean(acc) * 100) + "%")

**Mencoba meningkatkan akurasi terutama mencoba memprediksi beberapa film 'R'. Untuk ini akan melihat korelasi antara kolom dan label menggunakan chi2 independency test**

In [ ]:
movies = get_data().drop(['rating'], axis = 1)

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

for rating, category_id in sorted(category_to_id.items()):
    print("######### '{}':".format(rating))
    for col in movies.columns :
        features = tfidf.fit_transform(movies[col]).toarray()
        features_chi2 = chi2(features, labels == category_id)
        mean = np.mean(features_chi2[0])
        print("    '{}':".format(col))
        print("        {}".format(mean*100))

In [ ]:
movies = get_data()
#Group all the columns together
movies ['txt'] = movies['director'] + " " + movies['listed_in'] + " " + movies['type'] + " " + movies['country'] + " " + movies['cast'] 
movies = movies[['rating', 'txt']]


#Cross val
mskf = KFold(n_splits = 15, random_state=0, shuffle = True)
acc = []

for train_index, test_index in mskf.split(movies['txt']) :
    X_train, X_test = movies['txt'].iloc[train_index], movies['txt'].iloc[test_index]
    y_train, y_test = movies['rating'].iloc[train_index], movies['rating'].iloc[test_index]
    
    #Preprocess
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    
    #Fit
    clf = MultinomialNB().fit(X_train_tfidf, y_train)
    
    #score
    acc.append(clf.score(count_vect.transform(X_test.values), y_test.values))
    
plot_confusion_matrix(clf, count_vect.transform(X_test), np.array(y_test.values))
plt.show()

print("Accuracy with Naive Bayes on the whole dataset: " + str(clf.score(count_vect.transform(movies['txt'].values), movies['rating'].values) * 100) + "%")
    

# Konten Berdasarkan Rekomendasi Sistem
**Menggunakan Cosine Similarity**

In [ ]:
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
import nltk
import re

In [ ]:
data['director'] = data['director'].fillna('')
data['cast'] = data['cast'].fillna('')
data['text'] = data['title'] + ' '+data['director'] + ' '+ data['cast']+ ' ' +data['listed_in'] + ' '+data['description']

def preprocess(text):
    text = re.sub('[^A-z]', ' ', text)
    stop_words = set(stopwords.words('english'))  
    word_tokens = word_tokenize(text)  
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    filtered_sentence = []  
    for w in word_tokens:  
        if w not in stop_words:  
            filtered_sentence.append(lemmatizer.lemmatize(w))
    filtered = ' '.join([x for x in filtered_sentence])
    return filtered.lower().strip()
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
text_features = vectorizer.fit_transform(data['text'])
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(text_features)    

In [ ]:
def get_recommendation(movie_name):
        movie_index = data[data['title'] == movie_name].index
        movie_similarity = similarity_matrix[movie_index]
        movie_data = pd.DataFrame({'cosine_similarity':movie_similarity[0], 'index':np.arange(6234)})
        movie_data = movie_data.sort_values(by = 'cosine_similarity', ascending = False)
        topn=10
        movie_ids = movie_data['index'][1:topn]
        recommendation_movies = list()
        for temp in movie_ids:
            movie = data['title'][temp]
            recommendation_movies.append(movie)
        return  recommendation_movies
        
get_recommendation('Transformers: Robots in Disguise')